<a href="https://colab.research.google.com/github/salsaimon/proyecto_recursoshumanos/blob/main/preprocesamientoSQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#  pandas actualizado correctamente
!pip install --upgrade pandas

# importar pandas y sqlite3
import pandas as pd
import sqlite3 as sql

# Crear una conexión a la base de datos SQLite
conn = sql.connect('/content/drive/MyDrive/analitica3win/proyecto_recursoshumanos/base.db')

# Consultar las primeras filas de cada tabla
print("\nDatos de la tabla 'retiros':")
df_retiros = pd.read_sql("SELECT * FROM retiros LIMIT 5", conn)
print(df_retiros)

print("\nDatos de la tabla 'general':")
df_general = pd.read_sql("SELECT * FROM general LIMIT 5", conn)
print(df_general)

print("\nDatos de la tabla 'employee':")
df_employee = pd.read_sql("SELECT * FROM employee LIMIT 5", conn)
print(df_employee)

print("\nDatos de la tabla 'manager':")
df_manager = pd.read_sql("SELECT * FROM manager LIMIT 5", conn)
print(df_manager)

# Realizar consultas SQL para análisis
print("\nConteo de 'Attrition' en la tabla 'retiros':")
attrition_count = pd.read_sql("SELECT Attrition, COUNT(*) FROM retiros GROUP BY Attrition", conn)
print(attrition_count)

print("\nConteo de 'BusinessTravel' en la tabla 'general':")
business_travel_count = pd.read_sql("SELECT BusinessTravel, COUNT(*) FROM general GROUP BY BusinessTravel", conn)
print(business_travel_count)

# Realizar más consultas según sea necesario
# Ejemplo: Conteo de 'JobSatisfaction' en la tabla 'employee'
print("\nConteo de 'JobSatisfaction' en la tabla 'employee':")
job_satisfaction_count = pd.read_sql("SELECT JobSatisfaction, COUNT(*) FROM employee GROUP BY JobSatisfaction", conn)
print(job_satisfaction_count)



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 57.4 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.1.4
    Uninstalling pandas-2.1.4:
      Successfully uninstalled pandas-2.1.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pandas<2.2.2dev0,>=2.0, but you have pandas 2.2.2 which is incompatible.
google-colab 1.0.0 requires pandas==2.1.4, but you have pandas 2.2.2 which is incompatible.

Datos de la tabla 'retiros':
   index  EmployeeID Attrition       retirementDate retirementType  \
0      0           2       Yes  2016-10-15 00:00:00    Resignation   
1      1           7       Yes  2016-07-20 00:00:00    Resignation   
2      2          14       Yes  2016-11-08 00:00:00    Resignation   
3      3          29       Yes  2016-10-15 00:00:00    Resignation   
4      4          31   

In [ ]:
# Obtener la lista de tablas en la base de datos
tables = [row[0] for row in conn.execute("SELECT name FROM sqlite_master WHERE type='table'").fetchall()]

# Mostrar información sobre cada tabla y sus columnas
for table in tables:
    print(f"\nInformación sobre la tabla '{table}':")

    # Consultar las primeras filas de la tabla
    df = pd.read_sql(f"SELECT * FROM {table} LIMIT 5", conn)
    print(f"\nPrimeras 5 filas de la tabla '{table}':")
    print(df)

    # Mostrar la estructura de la tabla (tipos de datos)
    print(f"\nEstructura de la tabla '{table}':")
    cur = conn.cursor()
    cur.execute(f"PRAGMA table_info({table})")
    structure = cur.fetchall()
    for column in structure:
        print(f"Columna: {column[1]}, Tipo: {column[2]}")



Información sobre la tabla 'retiros':

Primeras 5 filas de la tabla 'retiros':
   index  EmployeeID Attrition       retirementDate retirementType  \
0      0           2       Yes  2016-10-15 00:00:00    Resignation   
1      1           7       Yes  2016-07-20 00:00:00    Resignation   
2      2          14       Yes  2016-11-08 00:00:00    Resignation   
3      3          29       Yes  2016-10-15 00:00:00    Resignation   
4      4          31       Yes  2016-02-08 00:00:00    Resignation   

  resignationReason  
0            Others  
1            Stress  
2            Others  
3            Others  
4            Salary  

Estructura de la tabla 'retiros':
Columna: index, Tipo: INTEGER
Columna: EmployeeID, Tipo: INTEGER
Columna: Attrition, Tipo: TEXT
Columna: retirementDate, Tipo: TIMESTAMP
Columna: retirementType, Tipo: TEXT
Columna: resignationReason, Tipo: TEXT

Información sobre la tabla 'general':

Primeras 5 filas de la tabla 'general':
   index  Age     BusinessTravel        

In [ ]:
## tratamiento de la tabla de retiros
# Contar registros por ID de empleado en la tabla de retiros
employee_id_count = pd.read_sql("SELECT EmployeeID, COUNT(*) as count FROM retiros GROUP BY EmployeeID", conn)

# Mostrar el resultado
print("\nConteo de registros por ID de empleado en la tabla 'retiros':")
print(employee_id_count)






Conteo de registros por ID de empleado en la tabla 'retiros':
     EmployeeID  count
0             2      1
1             7      1
2            14      1
3            29      1
4            31      1
..          ...    ...
706        4382      1
707        4387      1
708        4389      1
709        4392      1
710        4403      1

[711 rows x 2 columns]


In [ ]:
# Resumen de la columna 'retirementDate'
print("\nResumen de la columna 'retirementDate' en la tabla 'retiros':")

# Contar valores no nulos y nulos
total_retiros = pd.read_sql("SELECT COUNT(*) FROM retiros WHERE retirementDate IS NOT NULL", conn).iloc[0, 0]
total_nulos = pd.read_sql("SELECT COUNT(*) FROM retiros WHERE retirementDate IS NULL", conn).iloc[0, 0]
print(f"Total de fechas de retiro no nulas: {total_retiros}")
print(f"Total de fechas de retiro nulas: {total_nulos}")

# Estadísticas descriptivas
retiros_df = pd.read_sql("SELECT retirementDate FROM retiros WHERE retirementDate IS NOT NULL", conn)
retiros_df['retirementDate'] = pd.to_datetime(retiros_df['retirementDate'])  # Convertir a datetime
print("\nEstadísticas descriptivas de 'retirementDate':")
print(f"Fecha mínima de retiro: {retiros_df['retirementDate'].min()}")
print(f"Fecha máxima de retiro: {retiros_df['retirementDate'].max()}")

# Distribución por año
retiros_df['Año'] = retiros_df['retirementDate'].dt.year
retiros_por_anio = retiros_df.groupby('Año').size().reset_index(name='Cantidad de retiros')
print("\nDistribución de retiros por año:")
print(retiros_por_anio)




Resumen de la columna 'retirementDate' en la tabla 'retiros':
Total de fechas de retiro no nulas: 711
Total de fechas de retiro nulas: 0

Estadísticas descriptivas de 'retirementDate':
Fecha mínima de retiro: 2015-01-03 00:00:00
Fecha máxima de retiro: 2016-12-16 00:00:00

Distribución de retiros por año:
    Año  Cantidad de retiros
0  2015                   61
1  2016                  650

Distribución de retiros por mes:
        Mes  Cantidad de retiros
0   2015-01                    5
1   2015-02                    7
2   2015-03                    2
3   2015-04                    4
4   2015-05                    7
5   2015-06                    8
6   2015-07                    4
7   2015-08                    4
8   2015-09                    7
9   2015-10                    8
10  2015-11                    4
11  2015-12                    1
12  2016-01                   66
13  2016-02                   61
14  2016-03                   59
15  2016-04                   48
16  2016-0

In [ ]:
# Eliminar los registros con fechas de retiro en el año 2015
conn.execute("DELETE FROM retiros WHERE strftime('%Y', retirementDate) = '2015'")
conn.commit()

# Verificar que los registros han sido eliminados
retiros_df = pd.read_sql("SELECT retirementDate FROM retiros WHERE retirementDate IS NOT NULL", conn)
retiros_df['retirementDate'] = pd.to_datetime(retiros_df['retirementDate'])  # Convertir a datetime
print("\nVerificación después de eliminar los datos del año 2015:")
print(f"Fecha mínima de retiro: {retiros_df['retirementDate'].min()}")
print(f"Fecha máxima de retiro: {retiros_df['retirementDate'].max()}")

# Distribución de retiros por año después de eliminar los datos
retiros_df['Año'] = retiros_df['retirementDate'].dt.year
retiros_por_anio = retiros_df.groupby('Año').size().reset_index(name='Cantidad de retiros')
print("\nDistribución de retiros por año después de eliminación:")
print(retiros_por_anio)


Verificación después de eliminar los datos del año 2015:
Fecha mínima de retiro: 2016-01-02 00:00:00
Fecha máxima de retiro: 2016-12-16 00:00:00

Distribución de retiros por año después de eliminación:
    Año  Cantidad de retiros
0  2016                  650


In [ ]:
##analisis de retirementType

# Consultar los datos de la columna 'retirementType'
retiros_df = pd.read_sql("SELECT retirementType FROM retiros", conn)

# Contar valores únicos y frecuencia de cada tipo
print("\nResumen de la columna 'retirementType':")
print(f"Valores únicos en 'retirementType': {retiros_df['retirementType'].nunique()}")
print("\nFrecuencia de cada tipo de retiro:")
print(retiros_df['retirementType'].value_counts())

# Contar valores nulos
total_nulos = retiros_df['retirementType'].isnull().sum()
print(f"\nTotal de valores nulos en 'retirementType': {total_nulos}")


Resumen de la columna 'retirementType':
Valores únicos en 'retirementType': 2

Frecuencia de cada tipo de retiro:
retirementType
Resignation    585
Fired           65
Name: count, dtype: int64

Total de valores nulos en 'retirementType': 0


In [ ]:
import pandas as pd

# Leer todos los datos de la tabla 'retiros'
retiros_df = pd.read_sql("SELECT * FROM retiros", conn)

# Crear una copia del DataFrame original para modificaciones
retiros_processed_df = retiros_df.copy()

# Codificación One-Hot para la columna 'retirementType' sin eliminar ninguna categoría
encoded_retiros_df = pd.get_dummies(retiros_processed_df['retirementType'], prefix='retirementType')

# Convertir los valores True/False a 0/1
encoded_retiros_df = encoded_retiros_df.astype(int)

# Agregar las columnas codificadas al DataFrame procesado
retiros_processed_df = pd.concat([retiros_processed_df, encoded_retiros_df], axis=1)

# Eliminar la columna original 'retirementType' si ya no es necesaria
retiros_processed_df.drop('retirementType', axis=1, inplace=True)

# Imprimir el DataFrame procesado
print(retiros_processed_df.head())

# Mostrar el tamaño del DataFrame procesado
print(f"\nNúmero total de filas en el DataFrame procesado: {len(retiros_processed_df)}")


   index  EmployeeID Attrition       retirementDate resignationReason  \
0      0           2       Yes  2016-10-15 00:00:00            Others   
1      1           7       Yes  2016-07-20 00:00:00            Stress   
2      2          14       Yes  2016-11-08 00:00:00            Others   
3      3          29       Yes  2016-10-15 00:00:00            Others   
4      4          31       Yes  2016-02-08 00:00:00            Salary   

   retirementType_Fired  retirementType_Resignation  
0                     0                           1  
1                     0                           1  
2                     0                           1  
3                     0                           1  
4                     0                           1  

Número total de filas en el DataFrame procesado: 650


In [ ]:

# Leer todos los datos de la tabla 'retiros'
retiros_df = pd.read_sql("SELECT resignationReason FROM retiros", conn)

# Resumen de la columna 'resignationReason'
unique_reasons = retiros_df['resignationReason'].unique()
reason_counts = retiros_df['resignationReason'].value_counts()
missing_values = retiros_df['resignationReason'].isnull().sum()

# Imprimir resumen
print(f"\nValores únicos en 'resignationReason': {unique_reasons}")
print("\nFrecuencia de cada motivo de renuncia:")
print(reason_counts)
print(f"\nNúmero total de valores nulos en 'resignationReason': {missing_values}")


Valores únicos en 'resignationReason': ['Others' 'Stress' 'Salary' None]

Frecuencia de cada motivo de renuncia:
resignationReason
Others    296
Salary    173
Stress    116
Name: count, dtype: int64

Número total de valores nulos en 'resignationReason': 65


In [ ]:
# Reemplazar valores nulos con 'Unknown'
retiros_processed_df['resignationReason'].fillna('Unknown', inplace=True)

# Codificación One-Hot para la columna 'resignationReason'
encoded_reasons_df = pd.get_dummies(retiros_processed_df['resignationReason'], prefix='resignationReason', drop_first=False)

# Convertir valores True/False a 1/0 en las columnas codificadas
encoded_reasons_df = encoded_reasons_df.astype(int)

# Agregar las columnas codificadas al DataFrame procesado
retiros_processed_df = pd.concat([retiros_processed_df, encoded_reasons_df], axis=1)

# Eliminar la columna original 'resignationReason'
retiros_processed_df.drop('resignationReason', axis=1, inplace=True)

# Eliminar la columna 'Unknown' resultante de la codificación
if 'resignationReason_Unknown' in retiros_processed_df.columns:
    retiros_processed_df.drop('resignationReason_Unknown', axis=1, inplace=True)

# Imprimir el DataFrame procesado
print(retiros_processed_df.head())

# Mostrar el tamaño del DataFrame procesado
print(f"\nNúmero total de filas en el DataFrame procesado: {len(retiros_processed_df)}")

KeyError: 'resignationReason'

In [ ]:
# Imprimir las primeras filas del DataFrame procesado
print("Primeras filas de retiros_processed_df:")
print(retiros_processed_df.head())

# Mostrar el resumen de las nuevas columnas de codificación One-Hot
print("\nResumen de las columnas codificadas para 'resignationReason':")
encoded_columns = [col for col in retiros_processed_df.columns if col.startswith('resignationReason')]
print(retiros_processed_df[encoded_columns].sum())

# Mostrar el número total de filas en el DataFrame procesado
print(f"\nNúmero total de filas en el DataFrame procesado: {len(retiros_processed_df)}")

Primeras filas de retiros_processed_df:
   index  EmployeeID Attrition       retirementDate resignationReason  \
0      0           2       Yes  2016-10-15 00:00:00            Others   
1      1           7       Yes  2016-07-20 00:00:00            Stress   
2      2          14       Yes  2016-11-08 00:00:00            Others   
3      3          29       Yes  2016-10-15 00:00:00            Others   
4      4          31       Yes  2016-02-08 00:00:00            Salary   

   retirementType_Fired  retirementType_Resignation  
0                     0                           1  
1                     0                           1  
2                     0                           1  
3                     0                           1  
4                     0                           1  

Resumen de las columnas codificadas para 'resignationReason':
resignationReason    OthersStressOthersOthersSalaryOthersStressOthe...
dtype: object

Número total de filas en el DataFrame procesado:

In [ ]:
# Reemplazar valores nulos con 'Unknown'
retiros_processed_df['resignationReason'].fillna('Unknown', inplace=True)

# Codificación One-Hot para la columna 'resignationReason'
encoded_reasons_df = pd.get_dummies(retiros_processed_df['resignationReason'], prefix='resignationReason', drop_first=False)

# Agregar las columnas codificadas al DataFrame procesado
retiros_processed_df = pd.concat([retiros_processed_df, encoded_reasons_df], axis=1)

# Eliminar la columna original 'resignationReason'
retiros_processed_df.drop('resignationReason', axis=1, inplace=True)

# Eliminar la columna 'Unknown' resultante de la codificación
if 'resignationReason_Unknown' in retiros_processed_df.columns:
    retiros_processed_df.drop('resignationReason_Unknown', axis=1, inplace=True)

# Imprimir el DataFrame procesado
print(retiros_processed_df.head())

# Mostrar el tamaño del DataFrame procesado
print(f"\nNúmero total de filas en el DataFrame procesado: {len(retiros_processed_df)}")

KeyError: 'resignationReason'

In [ ]:
# Convertir valores True/False a 1/0 en las columnas codificadas
encoded_reasons_df = encoded_reasons_df.astype(int)

# Agregar las columnas codificadas al DataFrame procesado
retiros_processed_df = pd.concat([retiros_processed_df, encoded_reasons_df], axis=1)

# Eliminar la columna original 'resignationReason'
retiros_processed_df.drop('resignationReason', axis=1, inplace=True)

# Eliminar la columna 'Unknown' resultante de la codificación si existe
if 'resignationReason_Unknown' in retiros_processed_df.columns:
    retiros_processed_df.drop('resignationReason_Unknown', axis=1, inplace=True)

# Imprimir el DataFrame procesado
print(retiros_processed_df.head())

KeyError: "['resignationReason'] not found in axis"

In [ ]:
# Convertir valores True/False a 1/0 en las columnas especificadas
columns_to_convert = ['resignationReason_Others', 'resignationReason_Salary', 'resignationReason_Stress']
retiros_processed_df[columns_to_convert] = retiros_processed_df[columns_to_convert].astype(int)

# Imprimir el DataFrame procesado para verificar
print(retiros_processed_df.head())

   index  EmployeeID Attrition       retirementDate  retirementType_Fired  \
0      0           2       Yes  2016-10-15 00:00:00                     0   
1      1           7       Yes  2016-07-20 00:00:00                     0   
2      2          14       Yes  2016-11-08 00:00:00                     0   
3      3          29       Yes  2016-10-15 00:00:00                     0   
4      4          31       Yes  2016-02-08 00:00:00                     0   

   retirementType_Resignation  resignationReason_Others  \
0                           1                         1   
1                           1                         0   
2                           1                         1   
3                           1                         1   
4                           1                         0   

   resignationReason_Salary  resignationReason_Stress  \
0                         0                         0   
1                         0                         1   
2          

<ipython-input-32-835aef14ce2e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  retiros_processed_df[columns_to_convert] = retiros_processed_df[columns_to_convert].astype(int)


In [ ]:
# Resumen de la columna 'Attrition'
attrition_summary = retiros_processed_df['Attrition'].value_counts()
total_count = len(retiros_processed_df)

# Imprimir el resumen
print("Resumen de la variable 'Attrition':")
print(attrition_summary)
print(f"\nNúmero total de registros en 'Attrition': {total_count}")
print(f"Proporción de cada categoría en 'Attrition':")
print(attrition_summary / total_count)

Resumen de la variable 'Attrition':
Attrition
Yes    650
Name: count, dtype: int64

Número total de registros en 'Attrition': 650
Proporción de cada categoría en 'Attrition':
Attrition
Yes    1.0
Name: count, dtype: float64


In [ ]:
# Eliminar la columna 'Attrition' del DataFrame procesado
retiros_processed_df.drop('Attrition', axis=1, inplace=True)

# Imprimir el DataFrame procesado para verificar los cambios
print(retiros_processed_df.head())

# Mostrar el tamaño del DataFrame procesado
print(f"\nNúmero total de filas en el DataFrame procesado: {len(retiros_processed_df)}")

   index  EmployeeID       retirementDate  retirementType_Fired  \
0      0           2  2016-10-15 00:00:00                     0   
1      1           7  2016-07-20 00:00:00                     0   
2      2          14  2016-11-08 00:00:00                     0   
3      3          29  2016-10-15 00:00:00                     0   
4      4          31  2016-02-08 00:00:00                     0   

   retirementType_Resignation  resignationReason_Others  \
0                           1                         1   
1                           1                         0   
2                           1                         1   
3                           1                         1   
4                           1                         0   

   resignationReason_Salary  resignationReason_Stress  \
0                         0                         0   
1                         0                         1   
2                         0                         0   
3             

<ipython-input-35-bf67eeb8b32c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  retiros_processed_df.drop('Attrition', axis=1, inplace=True)


In [38]:
# Leer los datos de la tabla 'general'
general_df = pd.read_sql("SELECT * FROM general", conn)

# Imprimir las primeras filas del DataFrame
print("Primeras filas de la tabla 'general':")
print(general_df.head())

# Obtener información general sobre el DataFrame
print("\nInformación general de la tabla 'general':")
print(general_df.info())

# Resumen estadístico de las columnas numéricas
print("\nResumen estadístico de las columnas numéricas:")
print(general_df.describe())

# Resumen de las columnas categóricas
print("\nResumen de las columnas categóricas:")
for col in general_df.select_dtypes(include=['object']).columns:
    print(f"\nResumen de la columna '{col}':")
    print(general_df[col].value_counts())

Primeras filas de la tabla 'general':
   index  Age     BusinessTravel              Department  DistanceFromHome  \
0      0   51      Travel_Rarely                   Sales                 6   
1      1   31  Travel_Frequently  Research & Development                10   
2      2   32  Travel_Frequently  Research & Development                17   
3      3   38         Non-Travel  Research & Development                 2   
4      4   32      Travel_Rarely  Research & Development                10   

   Education EducationField  EmployeeCount  EmployeeID  Gender  ...  \
0          2  Life Sciences              1           1  Female  ...   
1          1  Life Sciences              1           2  Female  ...   
2          4          Other              1           3    Male  ...   
3          5  Life Sciences              1           4    Male  ...   
4          1        Medical              1           5    Male  ...   

   NumCompaniesWorked PercentSalaryHike StandardHours  StockOption

In [41]:
# Contar el número de registros por EmployeeID
employee_record_counts = general_df.groupby('EmployeeID').size()

# Mostrar los primeros registros de la cuenta por empleado
print(employee_record_counts.head())

EmployeeID
1    2
2    1
3    2
4    2
5    2
dtype: int64


In [62]:
# Crear una copia del DataFrame original
generalprocessed_df = general_df.copy()

# Agrupar por EmployeeID y calcular el promedio de las columnas numéricas
numeric_cols = generalprocessed_df.select_dtypes(include='number').columns

# Excluir 'EmployeeID' de las columnas numéricas para evitar conflictos
numeric_cols = numeric_cols.difference(['EmployeeID'])
avg_data = generalprocessed_df.groupby('EmployeeID')[numeric_cols].mean().reset_index()

# Mantener la información no numérica utilizando la moda
categorical_cols = generalprocessed_df.select_dtypes(include='object').columns

# Excluir 'EmployeeID' de las columnas categóricas
categorical_cols = categorical_cols.difference(['EmployeeID'])
mode_data = generalprocessed_df.groupby('EmployeeID')[categorical_cols].agg(lambda x: x.mode()[0] if not x.mode().empty else 'Unknown').reset_index()

# Unir los datos promediados con los datos categóricos
generalprocessed_df = avg_data.merge(mode_data, on='EmployeeID')

# Imprimir las primeras filas del nuevo DataFrame
print(generalprocessed_df.head())

# Mostrar información general del nuevo DataFrame
print(generalprocessed_df.info())

   EmployeeID   Age  DistanceFromHome  Education  EmployeeCount  JobLevel  \
0           1  51.0               6.0        2.0            1.0       1.0   
1           2  31.0              10.0        1.0            1.0       1.0   
2           3  32.0              17.0        4.0            1.0       4.0   
3           4  38.0               2.0        5.0            1.0       3.0   
4           5  32.0              10.0        1.0            1.0       1.0   

   MonthlyIncome  NumCompaniesWorked  PercentSalaryHike  StandardHours  ...  \
0       131160.0                 1.0               11.0            8.0  ...   
1        41890.0                 0.0               23.0            8.0  ...   
2       193280.0                 1.0               15.0            8.0  ...   
3        83210.0                 3.0               11.0            8.0  ...   
4        23420.0                 4.0               12.0            8.0  ...   

   YearsSinceLastPromotion  YearsWithCurrManager   index     B

In [63]:

# Verificar la cantidad de registros únicos por EmployeeID
print("Número único de EmployeeID:", generalprocessed_df['EmployeeID'].nunique())
print("Número total de registros en generalprocessed_df:", generalprocessed_df.shape[0])

# Verificar datos nulos
print("\nDatos nulos en generalprocessed_df:")
print(generalprocessed_df.isnull().sum())

# Revisar estadísticas descriptivas
print("\nResumen estadístico de generalprocessed_df:")
print(generalprocessed_df.describe(include='all'))

# Validar columnas de texto
print("\nConteo de valores únicos en columnas categóricas:")
print(generalprocessed_df[['BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole', 'MaritalStatus']].value_counts())

# Validar datos de tiempo
print("\nConteo de valores en InfoDate:")
print(generalprocessed_df['InfoDate'].value_counts())

# Comparar con datos originales (seleccionando algunos registros aleatorios)
print("\nComparación de registros aleatorios:")
sample_original = general_df.sample(5)
sample_processed = generalprocessed_df[generalprocessed_df['EmployeeID'].isin(sample_original['EmployeeID'])]

print("\nMuestra de datos originales:")
print(sample_original)
print("\nMuestra de datos procesados:")
print(sample_processed)

Número único de EmployeeID: 5060
Número total de registros en generalprocessed_df: 5060

Datos nulos en generalprocessed_df:
EmployeeID                  0
Age                         0
DistanceFromHome            0
Education                   0
EmployeeCount               0
JobLevel                    0
MonthlyIncome               0
NumCompaniesWorked         23
PercentSalaryHike           0
StandardHours               0
StockOptionLevel            0
TotalWorkingYears          11
TrainingTimesLastYear       0
YearsAtCompany              0
YearsSinceLastPromotion     0
YearsWithCurrManager        0
index                       0
BusinessTravel              0
Department                  0
EducationField              0
Gender                      0
InfoDate                    0
JobRole                     0
MaritalStatus               0
dtype: int64

Resumen estadístico de generalprocessed_df:
         EmployeeID          Age  DistanceFromHome    Education  \
count   5060.000000  5060.0000

In [64]:
from sklearn.impute import SimpleImputer

# Imputar valores nulos con la media
generalprocessed_df['NumCompaniesWorked'].fillna(generalprocessed_df['NumCompaniesWorked'].mean(), inplace=True)
generalprocessed_df['TotalWorkingYears'].fillna(generalprocessed_df['TotalWorkingYears'].mean(), inplace=True)

# Verificar si aún hay valores nulos
print("Datos nulos después de imputar:")
print(generalprocessed_df.isnull().sum())

Datos nulos después de imputar:
EmployeeID                 0
Age                        0
DistanceFromHome           0
Education                  0
EmployeeCount              0
JobLevel                   0
MonthlyIncome              0
NumCompaniesWorked         0
PercentSalaryHike          0
StandardHours              0
StockOptionLevel           0
TotalWorkingYears          0
TrainingTimesLastYear      0
YearsAtCompany             0
YearsSinceLastPromotion    0
YearsWithCurrManager       0
index                      0
BusinessTravel             0
Department                 0
EducationField             0
Gender                     0
InfoDate                   0
JobRole                    0
MaritalStatus              0
dtype: int64


<ipython-input-64-a01f3461165b>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  generalprocessed_df['NumCompaniesWorked'].fillna(generalprocessed_df['NumCompaniesWorked'].mean(), inplace=True)
<ipython-input-64-a01f3461165b>:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exa

In [65]:


# 1. Eliminar columnas no necesarias
columns_to_drop = ['EmployeeCount', 'StockOptionLevel', 'StandardHours']
generalprocessed_df = generalprocessed_df.drop(columns=columns_to_drop)

# 2. Estandarización de variables numéricas
numerical_columns = [
    'Age', 'DistanceFromHome', 'MonthlyIncome', 'NumCompaniesWorked',
    'PercentSalaryHike', 'TotalWorkingYears',
    'TrainingTimesLastYear', 'YearsAtCompany', 'YearsSinceLastPromotion',
    'YearsWithCurrManager'
]

# Inicializar el estandarizador
scaler = StandardScaler()

# Aplicar estandarización
generalprocessed_df[numerical_columns] = scaler.fit_transform(generalprocessed_df[numerical_columns])

# 3. Codificación de variables categóricas ordinales
ordinal_columns = {
    'Education': [1, 2, 3, 4, 5],
    'JobLevel': [1, 2, 3, 4, 5]
}

# Reemplazar categorías ordinales con valores numéricos
for col, order in ordinal_columns.items():
    generalprocessed_df[col] = pd.Categorical(generalprocessed_df[col], categories=order, ordered=True).codes

# 4. Codificación de variables categóricas no ordinales (One-Hot Encoding)
categorical_columns = [
    'BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole', 'MaritalStatus'
]

# Crear el codificador OneHotEncoder
one_hot_encoder = OneHotEncoder(sparse=False, drop='first')

# Aplicar el codificador a las columnas categóricas
one_hot_encoded = one_hot_encoder.fit_transform(generalprocessed_df[categorical_columns])

# Crear un DataFrame con las columnas codificadas
one_hot_df = pd.DataFrame(one_hot_encoded, columns=one_hot_encoder.get_feature_names_out(categorical_columns))

# Unir el DataFrame original con el DataFrame codificado
generalprocessed = pd.concat([generalprocessed_df.drop(categorical_columns, axis=1), one_hot_df], axis=1)

# Verificar el DataFrame final
print(generalprocessed.head())

   EmployeeID       Age  DistanceFromHome  Education  JobLevel  MonthlyIncome  \
0           1  1.565878         -0.394163          1         0       1.424580   
1           2 -0.591572          0.101202          0         0      -0.484657   
2           3 -0.483699          0.968092          3         3       2.753154   
3           4  0.163536         -0.889529          4         2       0.399063   
4           5 -0.483699          0.101202          0         0      -0.879679   

   NumCompaniesWorked  PercentSalaryHike  TotalWorkingYears  \
0           -0.684418          -1.156044          -1.273857   
1           -1.081723           2.116449          -0.629850   
2           -0.684418          -0.065213          -0.758651   
3            0.110192          -1.156044           0.271761   
4            0.507496          -0.883336          -0.243445   

   TrainingTimesLastYear  ...  JobRole_Human Resources  \
0               2.529894  ...                      0.0   
1               0.

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
